In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import pandas as pd

In [6]:

# Load the shapefile
landcover = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/landuse/gis_osm_landuse_a_free_1.shp")


In [11]:
# Filter by fclass
residential = landcover[landcover['fclass'] == 'residential'].copy()
farmland = landcover[landcover['fclass'] == 'farmland'].copy()
industrial = landcover[landcover['fclass'] == 'industrial'].copy()

In [8]:
df = pd.read_csv('dams.csv')

In [14]:
dam_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # Assuming WGS84
)

In [15]:
# Reproject everything to a metric CRS (e.g., UTM Zone 30N)
target_crs = "EPSG:32630"
dam_gdf = dam_gdf.to_crs(target_crs)
residential = residential.to_crs(target_crs)
farmland = farmland.to_crs(target_crs)
industrial = industrial.to_crs(target_crs)

In [16]:
# Helper function to compute nearest distance (in meters)
def nearest_distance(point, gdf_target):
    return gdf_target.geometry.distance(point).min()

In [17]:
# Apply distances
dam_gdf["dist_to_residential_m"] = dam_gdf.geometry.apply(lambda p: nearest_distance(p, residential))
dam_gdf["dist_to_farmland_m"] = dam_gdf.geometry.apply(lambda p: nearest_distance(p, farmland))
dam_gdf["dist_to_industrial_m"] = dam_gdf.geometry.apply(lambda p: nearest_distance(p, industrial))

In [19]:
# Save to CSV
dam_gdf[['longitude', 'latitude', 'dist_to_residential_m', 'dist_to_farmland_m', 'dist_to_industrial_m']].to_csv("dam_distances_to_landuse.csv", index=False)